# Decoradores de função e closures

Parte III - Funções como objetos | Capítulo 7 - Python Fluente - Luciano Ramalho


## Básico sobre decoradores

Um decorador é um **invocável** (callable) que aceita outra função como argumento (a função decorada).

```python
@decorate
def target():
    print('running target()')
```

```python
def target():
    print('running target()')

target = decorate(target)
```

Uma boa prática ṕe definir o decorador em um módulo e aplicar em funções em outros módulos.

In [ ]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [ ]:
@deco
def target():
    print('running target()')

In [ ]:
target()

running inner()


In [ ]:
target

<function __main__.deco.<locals>.inner()>

### Execução de decoradores

Os decoradores são executados imediatamente apóa a função decorada ser definida, ocorrendo em tempo de importação => **tempo de importação**.

As funções decoradas executam somente quando são explicitamente chamadas => **tempo de execução**.

In [ ]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

main()

running register(<function f1 at 0x7fdecb87b790>)
running register(<function f2 at 0x7fdecb87b310>)
running main()
registry -> [<function f1 at 0x7fdecb87b790>, <function f2 at 0x7fdecb87b310>]
running f1()
running f2()
running f3()


## Padrão Strategy melhorado com decorador

In [ ]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    """Seleciona o melhor desconto possível."""
    return max(promo(order) for promo in promos)

## Escopo de variáveis

Apesar da não exigência da declaração de variáveis, o interpretador Python supõe que uma variável cujo valor tenha sido atribuído no corpo de uma função é local.

In [ ]:
def mariana(a):
    print(a)
    print(b)

mariana(23)

23


NameError: ignored

In [ ]:
b = 40

mariana(23)

23
40


In [ ]:
b = 10

def lucas(a):
    print(a)
    print(b)
    b = 9

lucas(1)

1


UnboundLocalError: ignored

In [ ]:
b = 10

def lucas(a):
    global b
    print(a)
    print(b)
    b = 9

lucas(1)

1
10


In [ ]:
b = 2

In [ ]:
lucas(1)

1
2


### Comparando bytecodes

In [ ]:
from dis import dis

dis(mariana)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [ ]:
dis(lucas)

  5           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  6           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  7          16 LOAD_CONST               1 (9)
             18 STORE_GLOBAL             1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


## Closures

As closures são importantes quando temos funções aninhadas. Sua definição é uma função com um escopo estendido, que engloba variáveis não globais referenciadas no corpo da função que não estão definidas ali. Não importa se é anônima ou não, e sim se ela pode acessar variáveis não globais definidas fora de seu corpo.

In [5]:
"""
Uma classe para calcular uma média em evolução.
"""
class Averager():

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [7]:
avg = Averager()
avg

In [8]:
avg(10)

10.0

In [9]:
avg(11)

10.5

In [10]:
avg(12)

11.0

In [16]:
"""
Uma função de ordem superior para calcular uma média em evolução.
"""
def make_averager():

    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

In [17]:
avg = make_averager()
avg

<function __main__.make_averager.<locals>.averager(new_value)>

In [18]:
avg(10)

10.0

In [20]:
avg(11)

10.5

In [21]:
avg(12)

11.0

**Variável livre (free variable):** indica uma variável que não tem uma associação no escopo local.

In [19]:
avg()

TypeError: ignored